# 机械臂手眼标定-眼在手上-OpenCV
使用OpenCV内置的手眼标定API， 来完成手在眼上的手眼标定。

In [1]:
import numpy as np
import cv2
# 自定义库
from transform import Transform
from pose import Pose
from tsai import calibration_eye_in_hand

np.set_printoptions(precision=3, suppress=True)

## 原始数据

In [2]:
# 原始数据: 腕关节在机械臂基坐标系下的位姿 ^{arm}_{wrist}T
# 数据来源: 通过ABB六轴机械臂示教器读取
# 数据格式: x,y,z，roll,pitch,yaw
# xyz为三维坐标，单位m
# [roll, pitch, yaw] 为欧拉角, 单位°
raw_wrist_pose = np.float64([
    [0.82772, 0.06522, 0.75224, -144.93, 67.32, -145.78],
    [0.80865, 0.12002, 0.80706, 174.46, 58.62, 158.47],
    [0.80866, 0.12544, 0.67755, 110.68, 83.35, 89.47],
    [0.82286, 0.05224, 0.66590, -87.14, 70.66, -79.66],
    [0.88218,-0.01751, 0.75942, -131.85,60.81, -127.65],
    [0.87886, 0.02781, 0.80272, 148.30, 53.81, 152.58],
    [0.93914, 0.15168, 0.76964, 175.39, 66.25, 163.54],
    [0.92465, 0.19866, 0.72486, -152.84,75.11, -171.24],
    [0.91312, 0.14485, 0.72482, -106.60, 55.06, -124.55],
    [0.91310, 0.05494, 0.72477, -176.14, 74.58, -167.83],
    [0.92506, 0.08207, 0.80342, -129.92, 49.88, -136.51],
    [0.93684, 0.12112, 0.72428, 148.61, 71.30, 141.06]])

# 原始数据: 标定板在相机坐标系下的位姿 ^{cami}_{board}T
# 数据来源: 相机标定得到，相机外参。
# 数据格式: x,y,z,xn,yn,zn 
# xyz为三维坐标，单位m
# [xn, yn, zn]=rvect 旋转向量, 旋转角度为旋转向量的模长.
raw_caliboard_pose = np.float64([
    [-0.05278, -0.08160, 0.39593,  0.02834, -0.02854,  0.22926],
    [-0.13787, -0.07793, 0.37583,  0.24938, -0.22099, -0.15542],
    [-0.17817, -0.00343, 0.40412, -0.26167, -0.32992, -0.16666],
    [ 0.00962, -0.02616, 0.45339, -0.33369,  0.08834,  0.35973],
    [-0.08343, -0.09897, 0.34839,  0.03381,  0.03085,  0.40624],
    [-0.10354, -0.03125, 0.34939,  0.26487,  0.14562, -0.27929],
    [-0.04634, -0.02492, 0.27151,  0.12825, -0.16397, -0.09373],
    [-0.02279, -0.03672, 0.29697, -0.02907, -0.30641,  0.03926],
    [-0.01490, -0.06452, 0.29502, -0.01287, -0.36766,  0.50575],
    [-0.03314, -0.04779, 0.32411, -0.01859,  0.15740,  0.07058],
    [-0.04223, -0.06639, 0.27773,  0.19789, -0.16627,  0.49610],
    [-0.07012, -0.02279, 0.28054, -0.01190, -0.07518, -0.19488]])



## 构造旋转矩阵与平移向量

In [3]:
print("R_arm2wrist & t_arm2wrist_list")
# 腕关节在机械臂基坐标系下的位姿
R_arm2wrist_list = []
t_arm2wrist_list = []
T_arm2wrist_list = []
for x, y, z, roll, pitch, yaw in raw_wrist_pose:
    # 欧拉角转换为弧度
    rpy = np.radians(np.float64([roll, pitch, yaw]))
    # 旋转矩阵与平移向量
    R_arm2wrist = Transform.euler2rmat(*rpy)
    t_arm2wrist = np.float64([x, y, z])
    # 齐次变换矩阵
    T_arm2wrist = np.eye(4)
    T_arm2wrist[:3, :3] = R_arm2wrist
    T_arm2wrist[:3, 3] = t_arm2wrist
    # 添加到列表
    R_arm2wrist_list.append(R_arm2wrist)
    t_arm2wrist_list.append(t_arm2wrist)
    T_arm2wrist_list.append(T_arm2wrist)

R_arm2wrist & t_arm2wrist_list


In [4]:
print("R_cam2board_list & t_cam2board_list")
R_cam2board_list = []
t_cam2board_list = []
T_cam2board_list = []
for x, y, z, xn, yn, zn in raw_caliboard_pose:
    # 旋转向量转换为旋转矩阵
    R_cami2board = Transform.rvect2rmat([xn, yn, zn])
    t_cami2board = np.float64([x, y, z])
    # 齐次变换矩阵
    T_cam2board = np.eye(4)
    T_cam2board[:3, :3] = R_cami2board
    T_cam2board[:3, 3] = t_cami2board
    # 添加到列表
    R_cam2board_list.append(R_cami2board)
    t_cam2board_list.append(t_cami2board)
    T_cam2board_list.append(T_cam2board)

R_cam2board_list & t_cam2board_list


## OpenCV手眼标定

In [10]:
R_wrist2cam, t_wrist2cam = cv2.calibrateHandEye(R_arm2wrist_list, \
                    t_arm2wrist_list, R_cam2board_list, t_cam2board_list, \
                    method=cv2.CALIB_HAND_EYE_TSAI)
T_wrist2cam = np.eye(4)
T_wrist2cam[:3, :3] = R_wrist2cam
T_wrist2cam[:3, 3] = t_wrist2cam.reshape(-1)

print("T_wrist2cam")
print(T_wrist2cam)

T_wrist2cam
[[-0.021  1.     0.018 -0.115]
 [-1.    -0.021 -0.002  0.033]
 [-0.002 -0.018  1.    -0.028]
 [ 0.     0.     0.     1.   ]]


## 验证标定结果

In [7]:
n_sample = len(T_arm2wrist_list)
t_arm2board_list = []
for i in range(n_sample):
    T_arm2wristi = T_arm2wrist_list[i]
    T_cami2board = T_cam2board_list[i]
    T_arm2board = np.dot(np.dot(T_arm2wristi, T_wrist2cam), T_cami2board)
    t_arm2board = T_arm2board[:3, 3]
    print(f"标定板坐标[{i}]: {t_arm2board}")
    t_arm2board_list.append(t_arm2board)

标定板坐标[0]: [1.236 0.172 0.79 ]
标定板坐标[1]: [1.236 0.172 0.791]
标定板坐标[2]: [1.236 0.173 0.791]
标定板坐标[3]: [1.235 0.174 0.791]
标定板坐标[4]: [1.233 0.173 0.791]
标定板坐标[5]: [1.228 0.169 0.794]
标定板坐标[6]: [1.227 0.17  0.797]
标定板坐标[7]: [1.227 0.171 0.797]
标定板坐标[8]: [1.228 0.172 0.796]
标定板坐标[9]: [1.228 0.171 0.795]
标定板坐标[10]: [1.228 0.171 0.796]
标定板坐标[11]: [1.228 0.169 0.796]


In [8]:
t_arm2board_list = np.float64(t_arm2board_list)
# 计算均值
t_arm2board_mean = np.mean(t_arm2board_list, axis=0)
print(f"均值: {t_arm2board_mean}")
error_xyz = t_arm2board_list - t_arm2board_mean
print(f"各轴误差: {error_xyz}")
distance = np.linalg.norm(error_xyz, axis=1)
print(f"欧式距离: {distance}")

均值: [1.231 0.171 0.794]
各轴误差: [[ 0.006  0.001 -0.004]
 [ 0.005  0.    -0.003]
 [ 0.005  0.001 -0.002]
 [ 0.004  0.003 -0.003]
 [ 0.002  0.002 -0.003]
 [-0.003 -0.003  0.   ]
 [-0.004 -0.002  0.003]
 [-0.004 -0.     0.004]
 [-0.003  0.001  0.003]
 [-0.003 -0.001  0.001]
 [-0.003 -0.     0.002]
 [-0.003 -0.002  0.003]]
欧式距离: [0.007 0.006 0.006 0.006 0.004 0.004 0.005 0.005 0.004 0.003 0.003 0.005]


In [9]:
distance_mean = np.mean(distance)
print(f"distance_mean : {distance_mean}")

distance_mean : 0.00483045688515282


可以看到标定板在机械臂基坐标系下的位置误差， 最大是7mm， 均值为4.8mm。